# ISEN 370 Exam 3

# Question 1

In [8]:
# Q1
import pandas as pd

def silver_meal(requirements, setup_cost, holding_cost, initial_inventory):
    periods = len(requirements)
    i = 0
    ending_inventory = 0
    total_cost = 0
    detailed_plan = []
    
    # Adjust the first period's requirement based on initial inventory
    if periods > 0:
        effective_requirement = max(0, requirements[0] - initial_inventory)
        requirements = [effective_requirement] + requirements[1:]
    
    while i < periods:
        min_cost = float('inf')
        best_j = i
        cumulative_demand = 0
        
        for j in range(i, periods):
            cumulative_demand += requirements[j]
            period_cost = setup_cost + sum(holding_cost * (k - i) * requirements[k] for k in range(i+1, j+1))
            average_cost = period_cost / (j - i + 1)
            
            if average_cost > min_cost:
                break
            
            min_cost = average_cost
            best_j = j
        
        total_production = sum(requirements[i:best_j+1])
        
        if total_production > 0:
            total_cost += setup_cost  # Add setup cost once for the batch
            
        for k in range(i, best_j+1):
            if k == i:
                production_amount = total_production
            else:
                production_amount = 0
            
            inventory_cost = ending_inventory * holding_cost
            total_cost += inventory_cost
            ending_inventory += production_amount - requirements[k]
            ending_inventory = max(0, ending_inventory)
            detailed_plan.append((k + 1, requirements[k], production_amount, ending_inventory))
        
        i = best_j + 1
    
    df = pd.DataFrame(detailed_plan, columns=['Period', 'Requirements', 'Production', 'Ending Inventory'])
    return df, total_cost

# Example usage
requirements = [335,200,140,440,300,200]
setup_cost = 200
holding_cost = 0.3
initial_inventory = 0

print("Silver Meal Lot Sizing Heuristic:")
df, cost = silver_meal(requirements, setup_cost, holding_cost, initial_inventory)
print(df.to_string(index=False))
print("Total Cost: ", cost)


Silver Meal Lot Sizing Heuristic:
 Period  Requirements  Production  Ending Inventory
   1         335          675            340       
   2         200            0            140       
   3         140            0              0       
   4         440          940            500       
   5         300            0            200       
   6         200            0              0       
Total Cost:  754.0


# Question 2

In [2]:
# Q2

# Quesiton 3

In [12]:
# Q3

# Define the MPS for the end item
mps_end_item = {
    'Week': [6,7,8,9,10,11],
    'MPS': [220-75,165,180+30,120,75+30,300]
}

# Define product structure with predecessors
product_structure = {
    'Tires': {'Quantity': 4, 'Lead Time': 1, 'Level': 1, 'predecessor': 0},
    'Cat': {'Quantity': 1, 'Lead Time': 3, 'Level': 1, 'predecessor': 0}
}

# Function to recursively calculate lead time considering all predecessors
def calculate_total_lead_time(component, product_structure):
    lead_time = product_structure[component]['Lead Time']
    predecessor = product_structure[component]['predecessor']
    if predecessor == 0:
        return lead_time
    else:
        return lead_time + calculate_total_lead_time(predecessor, product_structure)

# Calculate planned order release for each component
def calculate_planned_order(component, mps_end_item, product_structure):
    planned_order = {}
    lead_time = product_structure[component]['Lead Time']
    level = product_structure[component]['Level']
    predecessor = product_structure[component]['predecessor']
    
    # Calculate start week based on total lead time considering all predecessors
    total_lead_time = calculate_total_lead_time(component, product_structure)
    start_week = mps_end_item['Week'][0] - total_lead_time
    #print(f"Start Week for Item {component}: {start_week}")
    
    for i in range(len(mps_end_item['Week'])):
        week = start_week + i
        net_requirement = mps_end_item['MPS'][i] * product_structure[component]['Quantity']
        
        # If the component has a predecessor, multiply net requirement by the product of quantities of all predecessors
        if predecessor:
            pred_quantity_product = 1
            pred = predecessor
            while pred:
                pred_quantity_product *= product_structure[pred]['Quantity']
                pred = product_structure[pred]['predecessor']
            net_requirement *= pred_quantity_product
        planned_order[week] = net_requirement

    return planned_order

# Calculate planned order release for each component
planned_order_release = {}
for component in product_structure:
    planned_order_release[component] = calculate_planned_order(component, mps_end_item, product_structure)

# Output the results
for component, planned_order in planned_order_release.items():
    print(f"Planned order release for component {component}:")
    print("Week\tPlanned order Release")
    for week, release in planned_order.items():
        print(week, "\t", release)
    print()


Planned order release for component Tires:
Week	Planned order Release
5 	 580
6 	 660
7 	 840
8 	 480
9 	 420
10 	 1200

Planned order release for component Cat:
Week	Planned order Release
3 	 145
4 	 165
5 	 210
6 	 120
7 	 105
8 	 300



# Question 4

In [17]:
# Q4
import pandas as pd

def silver_meal(requirements, setup_cost, holding_cost, initial_inventory):
    periods = len(requirements)
    i = 0
    ending_inventory = 0
    total_cost = 0
    detailed_plan = []
    
    # Adjust the first period's requirement based on initial inventory
    if periods > 0:
        effective_requirement = max(0, requirements[0] - initial_inventory)
        requirements = [effective_requirement] + requirements[1:]
    
    while i < periods:
        min_cost = float('inf')
        best_j = i
        cumulative_demand = 0
        
        for j in range(i, periods):
            cumulative_demand += requirements[j]
            period_cost = setup_cost + sum(holding_cost * (k - i) * requirements[k] for k in range(i+1, j+1))
            average_cost = period_cost / (j - i + 1)
            
            if average_cost > min_cost:
                break
            
            min_cost = average_cost
            best_j = j
        
        total_production = sum(requirements[i:best_j+1])
        
        if total_production > 0:
            total_cost += setup_cost  # Add setup cost once for the batch
            
        for k in range(i, best_j+1):
            if k == i:
                production_amount = total_production
            else:
                production_amount = 0
            
            inventory_cost = ending_inventory * holding_cost
            total_cost += inventory_cost
            ending_inventory += production_amount - requirements[k]
            ending_inventory = max(0, ending_inventory)
            detailed_plan.append((k + 1, requirements[k], production_amount, ending_inventory))
        
        i = best_j + 1
    
    df = pd.DataFrame(detailed_plan, columns=['Period', 'Requirements', 'Production', 'Ending Inventory'])
    return df, total_cost

# Example usage
requirements = [23,86,40,12]
setup_cost = 300
holding_cost = 3
initial_inventory = 0

print("Silver Meal Lot Sizing Heuristic:")
df, cost = silver_meal(requirements, setup_cost, holding_cost, initial_inventory)
print(df.to_string(index=False))
print("Total Cost: ", cost)


Silver Meal Lot Sizing Heuristic:
 Period  Requirements  Production  Ending Inventory
   1          23          161            138       
   2          86            0             52       
   3          40            0             12       
   4          12            0              0       
Total Cost:  906


In [18]:
import pandas as pd

def least_unit_cost(requirements, setup_cost, holding_cost, initial_inventory):
    periods = len(requirements)

    i = 0
    ending_inventory = 0
    total_cost = 0
    detailed_plan = []

    # Adjust the first period's requirement based on initial inventory
    if periods > 0:
        effective_requirement = max(0, requirements[0] - initial_inventory)
        requirements = [effective_requirement] + requirements[1:]

    while i < periods:
        min_cost = float('inf')
        best_j = i
        cumulative_demand = 0

        for j in range(i, periods):
            cumulative_demand += requirements[j]
            holding_costs = sum(holding_cost * (k - i) * requirements[k] for k in range(i+1, j+1))
            luc_cost = (setup_cost + holding_costs) / cumulative_demand

            if luc_cost > min_cost:
                break

            min_cost = luc_cost
            best_j = j

        total_production = sum(requirements[i:best_j+1])

        if total_production > 0:
            total_cost += setup_cost  # Add setup cost once for the batch

            for k in range(i, best_j+1):
                if k == i:
                    production_amount = total_production
                else:
                    production_amount = 0

                inventory_cost = ending_inventory * holding_cost
                total_cost += inventory_cost
                ending_inventory += production_amount - requirements[k]
                ending_inventory = max(0, ending_inventory)

                detailed_plan.append((k + 1, requirements[k], production_amount, ending_inventory))

        i = best_j + 1

    df = pd.DataFrame(detailed_plan, columns=['Period', 'Requirements', 'Production', 'Ending Inventory'])
    return df, total_cost

# Example usage
requirements = [23,86,40,12]
setup_cost = 300
holding_cost = 3
initial_inventory = 0

print("Least Unit Cost with Initial Inventory:")
df, cost = least_unit_cost(requirements, setup_cost, holding_cost, initial_inventory)
print(df.to_string(index=False))
print("Total Cost: ", cost)

Least Unit Cost with Initial Inventory:
 Period  Requirements  Production  Ending Inventory
   1          23          109             86       
   2          86            0              0       
   3          40           52             12       
   4          12            0              0       
Total Cost:  894


# Question 5

In [7]:
# Q5
import pandas as pd

class Task:
    def __init__(self, id, duration, due_date):
        self.id = id
        self.duration = duration
        self.due_date = due_date
        self.predecessors = []

def add_precedence_constraint(tasks, constraint):
    for i in range(len(constraint) - 1):
        task1_id, task2_id = constraint[i], constraint[i + 1]
        task1 = next(task for task in tasks if task.id == task1_id)
        task2 = next(task for task in tasks if task.id == task2_id)
        task2.predecessors.append(task1)

def lawler_algorithm(tasks):
    unassigned_tasks = tasks.copy()
    scheduled_tasks = []

    while unassigned_tasks:
        ready_tasks = [task for task in unassigned_tasks if all(pred in scheduled_tasks for pred in task.predecessors)]
        earliest_task = min(ready_tasks, key=lambda x: x.due_date)
        scheduled_tasks.append(earliest_task)
        unassigned_tasks.remove(earliest_task)

    return scheduled_tasks

def calculate_flow_time(task, scheduled_tasks):
    flow_time = 0
    for t in scheduled_tasks:
        if t.id <= task.id:
            flow_time += t.duration
    return flow_time

def calculate_tardiness(flow_time, due_date):
    tardiness = max(flow_time - due_date, 0)
    return tardiness

def main():
    jobs = [
        {'Job': '1', 'Processing time': 4, 'Due Date': 8},
        {'Job': '2', 'Processing time': 10, 'Due Date': 15},
        {'Job': '3', 'Processing time': 2, 'Due Date': 10},
        {'Job': '4', 'Processing time': 1, 'Due Date': 12},
        {'Job': '5', 'Processing time': 8, 'Due Date': 20}
    ]

    constraints = [('3','2','5')]

    tasks = [Task(job['Job'], job['Processing time'], job['Due Date']) for job in jobs]

    for constraint in constraints:
        add_precedence_constraint(tasks, constraint)

    scheduled_tasks = lawler_algorithm(tasks)

    data = []
    flow_time = 0
    for task in scheduled_tasks:
        flow_time += task.duration
        tardiness = calculate_tardiness(flow_time, task.due_date)
        data.append([task.id, task.duration, flow_time, task.due_date, tardiness])

    df = pd.DataFrame(data, columns=['Job', 'Processing Time', 'Flow Time', 'Due Date', 'Tardiness'])

    # Set display options to center align text
    pd.set_option('colheader_justify', 'center')
    pd.set_option('display.expand_frame_repr', False)

    print(df.to_string(index=False))

if __name__ == "__main__":
    main()


Job  Processing Time  Flow Time  Due Date  Tardiness
 1          4             4          8        0     
 3          2             6         10        0     
 4          1             7         12        0     
 2         10            17         15        2     
 5          8            25         20        5     


# Question 6

In [6]:
# Q6
import pandas as pd

class Task:
    def __init__(self, id, duration, due_date):
        self.id = id
        self.duration = duration
        self.due_date = due_date
        self.predecessors = []

def add_precedence_constraint(tasks, constraint):
    for i in range(len(constraint) - 1):
        task1_id, task2_id = constraint[i], constraint[i + 1]
        task1 = next(task for task in tasks if task.id == task1_id)
        task2 = next(task for task in tasks if task.id == task2_id)
        task2.predecessors.append(task1)

def lawler_algorithm(tasks):
    unassigned_tasks = tasks.copy()
    scheduled_tasks = []

    while unassigned_tasks:
        ready_tasks = [task for task in unassigned_tasks if all(pred in scheduled_tasks for pred in task.predecessors)]
        earliest_task = min(ready_tasks, key=lambda x: x.due_date)
        scheduled_tasks.append(earliest_task)
        unassigned_tasks.remove(earliest_task)

    return scheduled_tasks

def calculate_flow_time(task, scheduled_tasks):
    flow_time = 0
    for t in scheduled_tasks:
        if t.id <= task.id:
            flow_time += t.duration
    return flow_time

def calculate_tardiness(flow_time, due_date):
    tardiness = max(flow_time - due_date, 0)
    return tardiness

def main():
    jobs = [
        {'Job': '1', 'Processing time': 3, 'Due Date': 4},
        {'Job': '2', 'Processing time': 6, 'Due Date': 8},
        {'Job': '3', 'Processing time': 8, 'Due Date': 12},
        {'Job': '4', 'Processing time': 4, 'Due Date': 15},
        {'Job': '5', 'Processing time': 2, 'Due Date': 11},
        {'Job': '6', 'Processing time': 1, 'Due Date': 25},
        {'Job': '7', 'Processing time': 7, 'Due Date': 21}
    ]

    constraints = [()]

    tasks = [Task(job['Job'], job['Processing time'], job['Due Date']) for job in jobs]

    for constraint in constraints:
        add_precedence_constraint(tasks, constraint)

    scheduled_tasks = lawler_algorithm(tasks)

    data = []
    flow_time = 0
    for task in scheduled_tasks:
        flow_time += task.duration
        tardiness = calculate_tardiness(flow_time, task.due_date)
        data.append([task.id, task.duration, flow_time, task.due_date, tardiness])

    df = pd.DataFrame(data, columns=['Job', 'Processing Time', 'Flow Time', 'Due Date', 'Tardiness'])

    # Set display options to center align text
    pd.set_option('colheader_justify', 'center')
    pd.set_option('display.expand_frame_repr', False)

    print(df.to_string(index=False))

if __name__ == "__main__":
    main()


Job  Processing Time  Flow Time  Due Date  Tardiness
 1         3              3          4        0     
 2         6              9          8        1     
 5         2             11         11        0     
 3         8             19         12        7     
 4         4             23         15        8     
 7         7             30         21        9     
 6         1             31         25        6     


In [15]:
import pandas as pd

def FCFS(jobs):
    df = pd.DataFrame(jobs)
    df['Completion'] = df['Processing time'].cumsum()
    df['Tardiness'] = df.apply(lambda row: max(0, row['Completion'] - row['Due Date']), axis=1)
    mean_flow_time = df['Completion'].mean()
    avg_tardiness = df['Tardiness'].mean()
    num_tardy_jobs = df['Tardiness'].apply(lambda x: 1 if x > 0 else 0).sum()
    total_completion = df['Completion'].sum()
    total_tardiness = df['Tardiness'].sum()
    total_df = pd.DataFrame([{'Job': 'Total', 'Processing time': '-', 'Due Date': '-', 'Completion': total_completion, 'Tardiness': total_tardiness}])
    df = pd.concat([df, total_df], ignore_index=True).to_string(index=False)
    print(df)
    print("Mean flow time =", mean_flow_time)
    print("Average tardiness =", avg_tardiness)
    print("Number of tardy jobs =", num_tardy_jobs)
    return mean_flow_time, avg_tardiness, num_tardy_jobs

def SPT(jobs):
    df = pd.DataFrame(jobs)
    df = df.sort_values(by='Processing time')
    df['Completion'] = df['Processing time'].cumsum()
    df['Tardiness'] = df.apply(lambda row: max(0, row['Completion'] - row['Due Date']), axis=1)
    mean_flow_time = df['Completion'].mean()
    avg_tardiness = df['Tardiness'].mean()
    num_tardy_jobs = df['Tardiness'].apply(lambda x: 1 if x > 0 else 0).sum()
    total_completion = df['Completion'].sum()
    total_tardiness = df['Tardiness'].sum()
    total_df = pd.DataFrame([{'Job': 'Total', 'Processing time': '-', 'Due Date': '-', 'Completion': total_completion, 'Tardiness': total_tardiness}])
    df = pd.concat([df, total_df], ignore_index=True).to_string(index=False)
    print(df)
    print("Mean flow time =", mean_flow_time)
    print("Average tardiness =", avg_tardiness)
    print("Number of tardy jobs =", num_tardy_jobs)
    return mean_flow_time, avg_tardiness, num_tardy_jobs

def EDD(jobs):
    df = pd.DataFrame(jobs)
    df = df.sort_values(by='Due Date')
    df['Completion'] = df['Processing time'].cumsum()
    df['Tardiness'] = df.apply(lambda row: max(0, row['Completion'] - row['Due Date']), axis=1)
    mean_flow_time = df['Completion'].mean()
    avg_tardiness = df['Tardiness'].mean()
    num_tardy_jobs = df['Tardiness'].apply(lambda x: 1 if x > 0 else 0).sum()
    total_completion = df['Completion'].sum()
    total_tardiness = df['Tardiness'].sum()
    total_df = pd.DataFrame([{'Job': 'Total', 'Processing time': '-', 'Due Date': '-', 'Completion': total_completion, 'Tardiness': total_tardiness}])
    df = pd.concat([df, total_df], ignore_index=True).to_string(index=False)
    print(df)
    print("Mean flow time =", mean_flow_time)
    print("Average tardiness =", avg_tardiness)
    print("Number of tardy jobs =", num_tardy_jobs)
    return mean_flow_time, avg_tardiness, num_tardy_jobs

def CR(jobs):
    df = pd.DataFrame(jobs)
    # Initialize current time
    current_time = 0
    # Initialize a list to store scheduled jobs
    scheduled_jobs = []
    # Initialize an empty list to store job scheduling details
    schedule_data = []
    # Iterate until all jobs are scheduled
    while len(scheduled_jobs) < len(df):
        # Compute the CR values for all jobs
        df['CR'] = (df['Due Date'] - current_time) / df['Processing time']
        # Filter out the jobs that are already scheduled
        unscheduled_jobs = df[~df['Job'].isin(scheduled_jobs)]
        if not unscheduled_jobs.empty:
            # Select the job with the minimum CR
            next_job = unscheduled_jobs.loc[unscheduled_jobs['CR'].idxmin()]
            # Schedule the selected job
            next_job_start_time = max(current_time, next_job['Due Date'] - next_job['Processing time'])
            next_job_end_time = next_job_start_time + next_job['Processing time']
            scheduled_jobs.append(next_job['Job'])
            # Update the current time
            current_time = next_job_end_time
            # Calculate tardiness
            tardiness = max(0, next_job_end_time - next_job['Due Date'])
            # Add scheduling details to the list
            schedule_data.append({'Job': next_job['Job'],
                                  'Processing time': next_job['Processing time'],
                                  'Due Date': next_job['Due Date'],
                                  'Completion': next_job_end_time,
                                  'Tardiness': tardiness})
    # Add total row to the list
    total_tardiness = sum(entry['Tardiness'] for entry in schedule_data)
    total_completion_time = sum(entry['Completion'] for entry in schedule_data[:-1])
    schedule_data.append({'Job': 'Total',
                          'Processing time': '-',
                          'Due Date': '-',
                          'Completion': total_completion_time,
                          'Tardiness': total_tardiness})
    # Convert the list of dictionaries to a DataFrame
    schedule_df = pd.DataFrame(schedule_data)
    # Calculate mean flow time
    mean_flow_time = schedule_df['Completion'][:-1].mean()
    # Calculate average tardiness
    avg_tardiness = schedule_df['Tardiness'][:-1].mean()
    # Calculate number of tardy jobs
    num_tardy_jobs = (schedule_df['Tardiness'][:-1] > 0).sum()
    schedule_df = pd.DataFrame(schedule_data).to_string(index=False)
    print(schedule_df)
    print("Mean flow time =", mean_flow_time)
    print("Average tardiness =", avg_tardiness)
    print("Number of tardy jobs =", num_tardy_jobs)
    return mean_flow_time, avg_tardiness, num_tardy_jobs

# Example jobs data
jobs = [
    {'Job': '1', 'Processing time': 3, 'Due Date': 4},
    {'Job': '2', 'Processing time': 6, 'Due Date': 8},
    {'Job': '3', 'Processing time': 8, 'Due Date': 12},
    {'Job': '4', 'Processing time': 4, 'Due Date': 15},
    {'Job': '5', 'Processing time': 2, 'Due Date': 11},
    {'Job': '6', 'Processing time': 1, 'Due Date': 25},
    {'Job': '7', 'Processing time': 7, 'Due Date': 21}
]

# FCFS
print("First Come First Served (FCFS):")
fcfs_metrics = FCFS(jobs)
print()

# SPT
print("Shortest Processing Time (SPT):")
spt_metrics = SPT(jobs)
print()

# EDD
print("Earliest Due Date (EDD):")
edd_metrics = EDD(jobs)
print()

# CR
print("Critical Ratio Order (CR):")
cr_metrics = CR(jobs)
print()

# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Rule': ['FCFS', 'SPT', 'EDD', 'CR'],
    'Mean Flow Time': [fcfs_metrics[0], spt_metrics[0], edd_metrics[0], cr_metrics[0]],
    'Average Tardiness': [fcfs_metrics[1], spt_metrics[1], edd_metrics[1], cr_metrics[1]],
    '# Tardy Jobs': [fcfs_metrics[2], spt_metrics[2], edd_metrics[2], cr_metrics[2]]
}).to_string(index=False)

print("\n\nSUMMARY:")
print(summary_df)


First Come First Served (FCFS):
 Job  Processing time Due Date  Completion  Tardiness
    1        3            4          3          0    
    2        6            8          9          1    
    3        8           12         17          5    
    4        4           15         21          6    
    5        2           11         23         12    
    6        1           25         24          0    
    7        7           21         31         10    
Total        -            -        128         34    
Mean flow time = 18.285714285714285
Average tardiness = 4.857142857142857
Number of tardy jobs = 5

Shortest Processing Time (SPT):
 Job  Processing time Due Date  Completion  Tardiness
    6        1           25          1          0    
    5        2           11          3          0    
    1        3            4          6          2    
    4        4           15         10          0    
    2        6            8         16          8    
    7        7           2

# Question 7

In [ ]:
# Q7
import pandas as pd

def critical_path_analysis(schedule):
    # Convert Immediate Predecessors string to list
    for task in schedule:
        if isinstance(task['Immediate Predecessors'], str):
            task['Immediate Predecessors'] = task['Immediate Predecessors'].split(',')

    # Calculate Earliest Start (ES) and Earliest Finish (EF) times
    for task in schedule:
        if task['Immediate Predecessors'] == 0:
            task['ES'] = 0
            task['EF'] = task['Time Required']
        else:
            predecessors = task['Immediate Predecessors']
            if isinstance(predecessors, str):
                predecessors = [predecessors]
            task['ES'] = max([t['EF'] for t in schedule if t['Task'] in predecessors])
            task['EF'] = task['ES'] + task['Time Required']

    # Calculate Latest Start (LS) and Latest Finish (LF) times
    for task in reversed(schedule):
        if task['Task'] == 'I':  # Set LF for the last task
            task['LF'] = task['EF']
            task['LS'] = task['LF'] - task['Time Required']
        else:
            successors = [t['Task'] for t in schedule if t.get('Immediate Predecessors') and task['Task'] in t['Immediate Predecessors']]
            if not successors:
                task['LF'] = task['EF']
                task['LS'] = task['LF'] - task['Time Required']
            else:
                task['LF'] = min([t['LS'] for t in schedule if t['Task'] in successors])
                task['LS'] = task['LF'] - task['Time Required']

    # Calculate Slack
    for task in schedule:
        task['Slack'] = task['LF'] - task['EF']

    # Convert to DataFrame
    df = pd.DataFrame(schedule)

    # Extract critical path
    critical_path = df[df['Slack'] == 0]

    # Calculate project duration
    project_duration = critical_path['Time Required'].sum()

    return df.to_string(index=False), critical_path['Task'].tolist(), project_duration

def format_critical_path(critical_path):
    path_str = "Critical Path: "
    for i, task in enumerate(critical_path):
        if i != 0:
            path_str += " -> "
        path_str += task
    return path_str

# Example schedule
schedule = [
    {'Task': 'A1', 'Time Required': 1.8, 'Immediate Predecessors': 0},
    {'Task': 'B1', 'Time Required': 0.4, 'Immediate Predecessors': 'A1'},
    {'Task': 'C1', 'Time Required': 2.1, 'Immediate Predecessors': 'B1'},
    {'Task': 'A2', 'Time Required': 1.3, 'Immediate Predecessors': 'C4'},
    {'Task': 'B2', 'Time Required': 0.5, 'Immediate Predecessors': 'A2'},
    {'Task': 'C2', 'Time Required': 1.1, 'Immediate Predecessors': 'B2'},
    {'Task': 'A3', 'Time Required': 1.9, 'Immediate Predecessors': 'C2'},
    {'Task': 'B3', 'Time Required': 1.1, 'Immediate Predecessors': 'A3'},
    {'Task': 'C3', 'Time Required': 0.9, 'Immediate Predecessors': 'B3'},
    {'Task': 'A4', 'Time Required': 0.7, 'Immediate Predecessors': 'C5'},
    {'Task': 'B4', 'Time Required': 1.4, 'Immediate Predecessors': 'A5'},
    {'Task': 'C4', 'Time Required': 1.4, 'Immediate Predecessors': 'B5'},
    {'Task': 'A5', 'Time Required': 1.0, 'Immediate Predecessors': 'C1'},
    {'Task': 'B5', 'Time Required': 1.6, 'Immediate Predecessors': 'A1'},
    {'Task': 'C5', 'Time Required': 1.7, 'Immediate Predecessors': 'B1'}
]

# Perform critical path analysis
result_df, critical_path, project_duration = critical_path_analysis(schedule)

# Display results
print("Critical Path Analysis Results:")
print(result_df)

print("\n" + format_critical_path(critical_path))
print("Project Duration:", project_duration)
